# Creating Tools

Tools are interfaces that an agent, chain, or LLM can use to interact with the world.

### Wiki Tool

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper 

In [13]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [14]:
wiki.name

'wikipedia'

### Website Tool

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vector_db=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A1AF5666B0>)

In [10]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")


In [11]:
retriever_tool.name

'langsmith_search'

### Arxiv Tool

In [12]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [15]:
tools=[wiki,arxiv,retriever_tool]

In [16]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Programming\\Projects\\Face Body Detection\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001A19040D900>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaE

# Query from the Tools created

Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [35]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama2")

In [36]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [37]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

### Agent Executer

In [38]:
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [41]:
agent_executer.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...

As a helpful assistant, I'm glad to provide information about LangSmith!

LangSmith is a language model development platform that enables developers to create and deploy their own custom language models. The platform provides a simple and user-friendly interface for designing and training language models, allowing developers to focus on building innovative applications without worrying about the underlying complexity of natural language processing (NLP).

Here are some key features of LangSmith:

1. Customizable: LangSmith allows developers to create custom language models tailored to their specific use cases. They can choose from a variety of pre-trained models or start from scratch and design their own models.
2. Easy to use: The platform provides an intuitive interface that makes it simple for developers to design, train, and deploy language models. Even those without extensive NLP experience can create powerful language models using LangSmit

{'input': 'Tell me about LangSmith',
 'output': "\nAs a helpful assistant, I'm glad to provide information about LangSmith!\n\nLangSmith is a language model development platform that enables developers to create and deploy their own custom language models. The platform provides a simple and user-friendly interface for designing and training language models, allowing developers to focus on building innovative applications without worrying about the underlying complexity of natural language processing (NLP).\n\nHere are some key features of LangSmith:\n\n1. Customizable: LangSmith allows developers to create custom language models tailored to their specific use cases. They can choose from a variety of pre-trained models or start from scratch and design their own models.\n2. Easy to use: The platform provides an intuitive interface that makes it simple for developers to design, train, and deploy language models. Even those without extensive NLP experience can create powerful language mode